# Generate Fringes from datasets

In this notebook we produce the fringes files from the raw QUBIC datasets.

They are saved into numpy files of the type: fringes_bs_17_21_2020-10-27_11.34.04.npy

## Status of the code:
There is clear room for improvement:
- intercalibration are not good. This is addressed in the notebook called Analyse-Fringes-Oct-2020.Rmd
- for the fringe construction: apparently the fringe patterns seems to disappear gradually with the number of cycles (see for example with fringe [49,53] and [17,21] in the last part of code (label: Try to improve method). This is not understood yet and is a big issue as for now it seems that integrating longer does not improve statistics. 

In [ ]:
from pylab import *
import os
import sys
import time
import pickle
from importlib import reload


# Specific science modules
import healpy as hp
import scipy
import glob
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sop
import pandas as pd

from matplotlib.backends.backend_pdf import PdfPages
import qubic
from qubic import selfcal_lib as sc
from qubicpack.utilities import Qubic_DataDir
from qubicpack import qubicpack as qp
from qubicpack.qubicfp import qubicfp
import qubic.fibtools as ft
from qubic import fringes_lib as fl

# Get a dictionary
basedir = Qubic_DataDir(datafile='instrument.py', )
print('basedir : ', basedir)
dictfilename = basedir + '/dicts/global_source_oneDet.dict'
d = qubic.qubicdict.qubicDict()
d.read_from_file(dictfilename)
q = qubic.QubicInstrument(d)

reload(sc)

# looping over TES and Datasets

In [ ]:
# ============== Get data ==============

#### 27/10/2020
#### Vtes = 5
global_dir = '/Volumes/LaCie/QubicData/Calib-TD/Fringes/2020-10-27/'
vtes = 5
eco = 1
out_dir = '/Users/hamilton/Qubic/Calib-TD/Fringes/Fringes_2020-10-27_Vtes_5_Eco_1'
datasets = np.sort(glob.glob(global_dir+'/*RF_switch*'))
equiv = [0,0,1,1,2,2,3,3,3,3,3,3,3,3]    

#### 28/10/2020
# global_dir = '/Volumes/LaCie/QubicData/Calib-TD/Fringes/2020-10-28/'
# ### Vtes=4
# datasets = np.sort(glob.glob(global_dir+'/*RF_switch_Vtes_4_*'))
# vtes = 4
# eco = 1
# equiv = [0,0,1,1,2,2,3,3]    
# out_dir = '/Users/hamilton/Qubic/Calib-TD/Fringes/Fringes_2020-10-28_Vtes_4_Eco_1'
# ### Vtes=3.5
# datasets = np.sort(glob.glob(global_dir+'/*RF_switch_Vtes_3.5_*'))
# vtes = 3.5
# eco = 1
# equiv = [0,0,1,1,2,2,3,3]    
# out_dir = '/Users/hamilton/Qubic/Calib-TD/Fringes/Fringes_2020-10-28_Vtes_3.5_Eco_1'
### Vtes=4 & No Eccosorb
# datasets = np.sort(glob.glob(global_dir+'/*RF_switch_NoEco_Vtes_4_*'))
# vtes = 4
# eco = 0
# equiv = [0,0,1,1,2,2,3,3]    
# out_dir = '/Users/hamilton/Qubic/Calib-TD/Fringes/Fringes_2020-10-28_Vtes_4_Eco_0'


horns = []
ncycles = []
wt = []
for ds in datasets:
    strs = ds.split('_')
    ncycles.append(float(strs[strs.index('ncycles')+1]))
    wt.append(float(strs[strs.index('wt')+1]))
    horns.append([int(strs[-2]), int(strs[-1])])

print('Ncycles')
print(ncycles)
print('WT')
print(wt)
print('Horns')
print(horns)

print()
all_equiv = np.unique(equiv)

print('equivalency of baselines')
print(equiv)
print()
print()
print()
print('Found {} equivalency classes:'.format(len(all_equiv)))
for i in range(len(all_equiv)):
    dsequiv = where(np.array(equiv)==all_equiv[i])[0]
    print(' - Type {}'.format(all_equiv[i]))
    for j in range(len(dsequiv)):
        print('     * dsnum={} [{},{}]'.format(dsequiv[j], horns[dsequiv[j]][0], horns[dsequiv[j]][1]))
   


In [ ]:
rc('figure', figsize=(16,7))
reload(sc)
for i in range(len(all_equiv)):
    dsequiv = where(np.array(equiv)==all_equiv[i])[0]
    subplot(1,4,i+1, aspect='equal')
    sc.plot_horns(q)
    title('Type = {}'.format(all_equiv[i]))
    print(' - Type {}'.format(all_equiv[i]))
    for j in range(len(dsequiv)):
        print('     * dsnum={} [{},{}]'.format(dsequiv[j], horns[dsequiv[j]][0], horns[dsequiv[j]][1]))
        sc.plot_baseline(q,[horns[dsequiv[j]][0], horns[dsequiv[j]][1]])
    legend(fontsize=5)

In [ ]:
# =================== Make a mask ==============
# Mask to remove the 8 thermometer pixels
mask = np.ones((17,17))
mask[0, 12:] = np.nan
mask[1:5, 16] = np.nan


In [ ]:
# ============ Analysis for both ASICs and all measurements ==================
rc('figure', figsize=(16,7))
reload(fl)

# select a TES
tes = 95

# Filter the data (just to give an idea because it is done when folding)
lowcut = 0.00001
highcut = 2.
nharm = 10
notch = np.array([[1.724, 0.005, nharm]])



### Check that out_dire exists, if not try to create
try:
    os.makedirs(out_dir, exist_ok = True)
except OSError as e:
    print('Unable to create Output Directory: '+out_dir)

########### The big loop
for myequiv in range(len(all_equiv)):

    mydsequiv = where(np.array(equiv)==myequiv)[0]
    print(mydsequiv)

    neq = len(mydsequiv)
    sgns = np.ones((len(mydsequiv),17,17))
    for i in range(neq):
        sgns[i,:,:] *= (-1)**i

    print('=============================================================================================')
    print('Type {}'.format(myequiv))
    for j in range(len(mydsequiv)):
            print('     * dsnum={} [{},{}]'.format(mydsequiv[j], horns[mydsequiv[j]][0], horns[mydsequiv[j]][1]))


    myw = None

    all_folded = []
    all_params = []
    all_res_louise = []
    all_periods = []
    all_names = ''
    all_bs_names = ''
    all_fringes = []
    all_pairs= []


    for ids in mydsequiv:
        print("    ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        print("    ++ "+datasets[ids])

        ### Dataset name
        name = datasets[ids].split('/')[-1].split('_')
        gen_name = 'fringes_bs_{}_{}_NCycles_{}_WT_{}_Vtes_{}_Eco_{}_'.format(horns[ids][0], 
                                                                              horns[ids][1], 
                                                                              ncycles[ids], wt[ids],
                                                                              vtes, eco)+name[0]+'_'+name[1]
        
        t, folded_bothasics, param_est, res_mich, res_louise, period = fl.analyse_fringes(datasets, ids, myw, 
                                                                            lowcut=lowcut, highcut=highcut, notch=notch,
                                                                            t0=None, tf=None, silent=True,
                                                                            stable_time=wt[ids]/1000)
        all_folded.append(folded_bothasics)
        all_params.append(param_est)
        all_res_louise.append(res_louise)
        all_periods.append(period)
        stable_time = period/6


        # ============== Plots ===========================================================
        # Look at one fit
        x0_est = param_est[tes - 1, 1]
        amps_est = param_est[tes - 1, 2:8]
        figure()
        subplot(1,2,1)
        plt.plot(t, folded_bothasics[tes - 1, :], label='folded signal')
        plt.plot(t, ft.simsig_fringes(t, stable_time, param_est[tes - 1, :]), label='fit')
        plt.plot(np.arange(0, 6 * stable_time, stable_time) + x0_est, amps_est, 'ro', label='amplitudes')
        for k in range(6): 
            plt.axvline(x=stable_time*k+ x0_est,color='k', ls=':',alpha=0.3)
        resid = folded_bothasics[tes - 1, :]-ft.simsig_fringes(t, stable_time, param_est[tes - 1, :])
        plt.plot(t, resid, label='Residuals: RMS={0:6.4f}'.format(np.std(resid)))
        name = datasets[mydsequiv[i]].split('/')[-1].split('_')
        title('Baseline {}'.format(horns[ids])+'\n'+name[0]+' '+name[1])
        legend(loc='upper right')
        ylim(-1,2.5)

        fringe_louise = ft.image_asics(all1=res_louise)
        fringe_louise /= np.nanstd(fringe_louise)
        all_fringes.append(fringe_louise)
        all_pairs.append(horns[ids])


        subplot(1,2,2)
        lim = 2
        plt.imshow(nan_to_num(fringe_louise * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
        ft.qgrid()
        name = datasets[ids].split('/')[-1].split('_')
        myname = name[0]+' '+name[1]+' ; '
        all_names = all_names + myname
        title('Baseline {}'.format(horns[ids])+'\n'+name[0]+' '+name[1])
        all_bs_names += '{}'.format(horns[ids])
        plt.colorbar()

        
        ### Save Image
        savefig(out_dir+'/'+gen_name+'.png')
        
#         #------------ Saving in a file
#         filename = gen_name+'.npy'
#         np.save(out_dir+'/'+filename, fringe_louise)
#         print('    ++ Fringe saved in file: '+out_dir+filename)
#         print("    ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        plt.show()

    #### Now plotting sum and difference
    av_fringe = np.sum(np.array(all_fringes), axis=0)/2
    diff_fringe = np.sum(np.array(all_fringes)*sgns, axis=0)/2
    lim = 2
    figure()
    subplot(1,2,1)
    plt.imshow(nan_to_num(av_fringe * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
    ft.qgrid()
    plt.title('Sum / {}'.format(neq)+'\n'+all_bs_names+'\n'+myname)
    plt.colorbar()
    subplot(1,2,2)
    plt.imshow(nan_to_num(diff_fringe * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
    ft.qgrid()
    plt.title('diff / {}'.format(neq)+'\n'+all_bs_names+'\n'+myname)
    plt.colorbar()
    tight_layout()      
    print('=============================================================================================')
    plt.show()


### Try to improve method

In [ ]:
reload(fl)
reload(sc)
reload(ft)
#### 27/10/2020
#### Vtes = 5
global_dir = '/Volumes/LaCie/QubicData/Calib-TD/Fringes/2020-10-27/'
datasets = np.sort(glob.glob(global_dir+'/*RF_switch*'))
equiv = [0,0,1,1,2,2,3,3,3,3,3,3,3,3]    

#### Read data for a given baseline
ids = 0
myds = [datasets[ids]]
stable_time = wt[ids]/1000
print(myds)

read_data = []
#### Read data
for asic in [1, 2]:
    _, my_t_data, my_data = fl.get_data(myds, 0, asic, doplot=False)
    read_data.append([my_t_data, my_data])
    tmin = my_t_data[0]
    tmax = my_t_data[-1]
    delta_t = tmax-tmin
print(tmin, tmax, delta_t)

In [ ]:
def doit(myds, read_data, lowcut, highcut, notch, t0, tf, stable_time, doplot=True, tes=95):
    myw=None
    t, folded_bothasics, param_est, res_mich, res_louise, myperiod = fl.analyse_fringes(myds, 0, myw, 
                                                                        lowcut=lowcut, highcut=highcut, notch=notch,
                                                                        t0=t0, tf=tf, 
                                                                        stable_time=stable_time, read_data=read_data)

    ### Residuals
    resid = np.zeros_like(folded_bothasics)
    amp_est_TES = np.zeros((256, 6))
    x0_est_TES = np.zeros(256)
    for ites in range(256):
        x0_est = param_est[ites, 1]
        x0_est_TES[ites] = x0_est
        amps_est = param_est[ites, 2:8]
        amp_est_TES[ites,:] = amps_est
        resid[ites,:] = folded_bothasics[ites, :]-ft.simsig_fringes(t, myperiod/6, param_est[ites, :])

    ### RMS residuals for each TES with clipping
    std_resid = np.zeros(256)
    for ites in range(256):
        mm, ss = ft.meancut(resid[ites,:],3)
        std_resid[ites] = ss
    m_std = np.mean(std_resid)
    s_std = np.std(std_resid)
    ###############

    ### Fringes ####
    fringe_louise = ft.image_asics(all1=res_louise)
    #fringe_louise /= np.nanstd(fringe_louise)
    ################
    
    ### Plot
    if doplot==True:
        figure()
        subplot(2,2,1)
        plt.plot(t, folded_bothasics[tes - 1, :], label='folded signal')
        plt.plot(t, ft.simsig_fringes(t, myperiod/6, param_est[tes - 1, :]), label='fit')
        plt.plot(np.arange(0, myperiod, myperiod/6) + x0_est_TES[tes-1], amp_est_TES[tes-1,:], 'ro', label='amplitudes')
        plt.plot(t, resid[tes-1,:], label='Residuals 1 (clipped): RMS={0:6.4f}'.format(std_resid[tes-1]))
        residuals = folded_bothasics[tes - 1, :] - ft.simsig_fringes(t, myperiod/6, param_est[tes - 1, :])
        plt.plot(t, t*0+std_resid[tes-1],'k--', alpha=0.3)
        plt.plot(t, t*0,'k:', alpha=0.3)
        plt.plot(t, t*0-std_resid[tes-1],'k--', alpha=0.3)
        plt.plot(t, t*0+res_louise[tes-1],'r:', label='Fringe value: {0:6.4f}'.format(res_louise[tes-1]))
        
        replot_data = [t, 
                       folded_bothasics[tes - 1, :], 
                       ft.simsig_fringes(t, myperiod/6, param_est[tes - 1, :]), 
                       resid[tes-1,:],
                       amp_est_TES[tes-1,:], 
                       x0_est_TES[tes-1],
                      res_louise[tes-1], myperiod, fringe_louise]
        
        legend(loc='upper right')
        title('TES #{}'.format(tes))
        ylim(-3,3)

        subplot(2,2,2)
        lim = 2
        plt.imshow(nan_to_num(fringe_louise * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
        ft.qgrid()
        colorbar()
        title('Fringe')

#         subplot(2,2,3)
#         h=hist(std_resid, bins=20, range=[0,1.5], label = 'Av RMS = {0:6.3f} +/- {1:6.3f}'.format(m_std, s_std))
#         legend()
#         title('All TES RMS')

#         subplot(2,2,4)
#         plt.imshow(nan_to_num(ft.image_asics(all1=std_resid) * mask), vmin=0, vmax=1.5, interpolation='Gaussian')
#         title('All TES RMS')
#         colorbar()
        
        tight_layout()
        plt.show()
        
    return fringe_louise, ft.image_asics(all1=std_resid), m_std, s_std, amp_est_TES, x0_est_TES, myperiod, replot_data
    


In [ ]:
reload(fl)

rc('figure', figsize=(16,10))

mytes = 93

lowcut = 0.00001
highcut = 2.
nharm = 10
notch = np.array([[1.724, 0.005, nharm]])


t0 = None
tf = None
full_fringe, full_std_resid, full_m_std, full_s_std, amp_est_TES, x0_est_TES,period, replot = doit(myds, read_data, 
                                                                                lowcut, highcut, notch, 
                                                                                t0, tf, stable_time, tes=mytes)

In [ ]:
### One can replot the above from outside the fct
def replot_one_tod(replot):
    plt.plot(replot[0], replot[1], label='folded signal')
    plt.plot(replot[0], replot[2], label='fit')
    mm, ss = ft.meancut(replot[3], 3)
    plt.plot(replot[0], replot[3], label='Residuals 1 (clipped): RMS={0:6.4f}'.format(ss))
    plt.plot(np.arange(0, replot[7], replot[7]/6) + replot[5], replot[4], 'ro', label='amps')
    plt.plot(replot[0], replot[0]*0+replot[6],'r:', label='Fringe value: {0:6.4f}'.format(replot[6]))
    plt.plot(replot[0], replot[0]*0,'k:', alpha=0.3)
    ylim(-3,3)
    xlim(-replot[7]/6, replot[7]+replot[7]/6)
    axvline(x=0,color='k', ls=':', alpha=0.3)
    axvline(x=replot[7],color='k', ls=':', alpha=0.3)
    legend(loc='upper right')
    title('TES #{}'.format(mytes))

def replot_one_asic(replot, mask):
    lim = 2
    plt.imshow(nan_to_num(replot[8] * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
    ft.qgrid()
    colorbar()
    title('Fringe')

    
    
rc('figure', figsize=(16,7))
subplot(1,2,1)
replot_one_tod(replot)
subplot(1,2,2)
replot_one_asic(replot, mask)
tight_layout()

In [ ]:
reload(fl)
print(tmin, tmax, delta_t)

split = (delta_t/period).astype(int)
print(split)

all_tmin = tmin + np.arange(split)*delta_t/split
all_tmax = all_tmin+delta_t/split

all_fringe = []
all_std_resid = []
all_m_std = []
all_s_std = []
all_amp_est_TES = []
all_x0_est_TES = []
all_periods = []
all_replot = []
for js in range(split):
    #js=2
    print('Doing',all_tmin[js], all_tmax[js])
    fringe, std_resid, m_std, s_std, amp_est_TES, x0_est_TES,theperiod, replot = doit(myds, read_data, 
                                                        lowcut, highcut, notch, 
                                                        all_tmin[js], all_tmax[js], stable_time, tes=mytes)
    all_fringe.append(fringe)
    all_std_resid.append(std_resid)
    all_m_std.append(m_std)
    all_s_std.append(s_std)
    all_amp_est_TES.append(amp_est_TES)
    all_x0_est_TES.append(x0_est_TES)
    all_periods.append(theperiod)
    all_replot.append(replot)

In [ ]:
### Available
all_tmin = np.array(all_tmin)
all_tmax = np.array(all_tmax)
all_fringe = np.array(all_fringe)
all_std_resid = np.array(all_std_resid)
all_m_std = np.array(all_m_std)
all_s_std = np.array(all_s_std)
all_amp_est_TES = np.array(all_amp_est_TES)
all_x0_est_TES = np.array(all_x0_est_TES)
all_periods = np.array(all_periods)

print(all_fringe.shape)
print(all_std_resid.shape)
print(all_m_std.shape)
print(all_s_std.shape)
print(all_amp_est_TES.shape)
print(all_x0_est_TES.shape)
print(all_periods.shape)

subplot(1,2,1)
plot(all_tmin, all_periods)
ylabel('Period')
xlabel('Time')

subplot(1,2,2)
plot(all_tmin,all_x0_est_TES[:,mytes-1])
ylabel('X0_est')
xlabel('Time')


In [ ]:
plt.ion()
rc('figure', figsize=(16,5))

### Brutal average
raw_av_fringe = np.mean(all_fringe,axis=0)
raw_err_fringe = np.std(all_fringe,axis=0)

lim = 1

figure()
subplot(1,3,1)
plt.imshow(nan_to_num(full_fringe * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
ft.qgrid()
colorbar()
title('Full Duration Fringe')
subplot(1,3,2)
plt.imshow(nan_to_num(full_fringe*0 * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
subplot(1,3,3)
plt.imshow(nan_to_num(full_fringe*0 * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')



figure()
subplot(1,3,1)
plt.imshow(nan_to_num(raw_av_fringe * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
ft.qgrid()
colorbar()
title('Brute Force Averaged Fringe - Mean')
subplot(1,3,2)
plt.imshow(nan_to_num(raw_err_fringe/np.sqrt(len(all_m_std)) * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
ft.qgrid()
colorbar()
title('Brute Force Averaged Fringe - Error')
subplot(1,3,3)
plt.imshow(nan_to_num((raw_av_fringe-raw_av_fringe) * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
ft.qgrid()
colorbar()
title('Diff with Brute Force')


# ### Average weighted by global RMS
global_av_fringe = np.zeros((17,17))
global_err_fringe = np.zeros((17,17))
for i in range(17):
    for j in range(17):
        global_av_fringe[i,j], global_err_fringe[i,j] = ft.weighted_mean(all_fringe[:,i,j], all_m_std)
        
figure()
subplot(1,3,1)
plt.imshow(nan_to_num(global_av_fringe * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
ft.qgrid()
colorbar()
title('Global Weighted Averaged Fringe - Mean')
subplot(1,3,2)
plt.imshow(nan_to_num(global_err_fringe/np.sqrt(len(all_m_std)) * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
ft.qgrid()
colorbar()
title('Global Weighted Averaged Fringe - Error')
subplot(1,3,3)
plt.imshow(nan_to_num((global_av_fringe-raw_av_fringe) * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
ft.qgrid()
colorbar()
title('Diff with Brute Force')



# ### Average weighted by RMS of each TES
local_av_fringe = np.zeros((17,17))
local_err_fringe = np.zeros((17,17))
for i in range(17):
    for j in range(17):
        local_av_fringe[i,j], local_err_fringe[i,j] = ft.weighted_mean(all_fringe[:,i,j], all_std_resid[:,i,j])
        
figure()
subplot(1,3,1)
plt.imshow(nan_to_num(local_av_fringe * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
ft.qgrid()
colorbar()
title('Each Weighted Averaged Fringe - Mean')
subplot(1,3,2)
plt.imshow(nan_to_num(local_err_fringe/np.sqrt(len(all_m_std)) * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
ft.qgrid()
colorbar()
title('Each TES Weighted Averaged Fringe - Error')
subplot(1,3,3)
plt.imshow(nan_to_num((local_av_fringe-raw_av_fringe) * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
ft.qgrid()
colorbar()
title('Diff with Brute Force')





In [ ]:
import matplotlib.animation
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ion()
fig, ax = plt.subplots()

nframes = len(all_m_std)

def animate(t):
    plt.cla()
    findex = t/nframes*(len(all_m_std)-1)
    index = int(floor(findex))
    delta = findex-index
    myimg0 = nan_to_num(all_fringe[index,:,:] * mask)
    myimg1 = nan_to_num(all_fringe[index+1,:,:] * mask)
    myimg = myimg0*(1-delta) + myimg1*delta
    plt.imshow(myimg, cmap='bwr', vmin=-1,vmax=1, interpolation='Gaussian')
    ft.qgrid()
    title('Index = {}'.format(index))

matplotlib.animation.FuncAnimation(fig, animate, frames=nframes)

In [ ]:
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ion()
fig, ax = plt.subplots()

nframes = len(all_m_std)

def animate(t):
    plt.cla()
    findex = t/nframes*(len(all_m_std)-1)
    index = int(floor(findex))
    replot_one_tod(all_replot[index])

matplotlib.animation.FuncAnimation(fig, animate, frames=nframes)

In [ ]:
rc('figure', figsize=(16,7))
tes = 95

t0 = None
tf = 180
fringe_cut, _,_,_,_,_,_,_ = doit(myds, read_data, lowcut, highcut, notch, t0, tf, stable_time, tes=mytes)

figure()
subplot(1,2,1)
lim = 2
plt.imshow(nan_to_num(full_fringe * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
ft.qgrid()
colorbar()
title('Fringe all times')

subplot(1,2,2)
lim = 2
plt.imshow(nan_to_num(fringe_cut * mask), vmin=-lim, vmax=lim, cmap='bwr', interpolation='Gaussian')
ft.qgrid()
colorbar()
title('Fringe t<{}'.format(tf))

